In [1]:
import os, time, pickle

import numpy as np
np.random.seed(42)

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Reading the data
folderName = '../data/' 
fileName   = 'yacht_hydrodynamics.data'

#Attribute Information:

# Variations concern hull geometry coefficients and the Froude number: 

# 1. Longitudinal position of the center of buoyancy, adimensional. 
# 2. Prismatic coefficient, adimensional. 
# 3. Length-displacement ratio, adimensional. 
# 4. Beam-draught ratio, adimensional. 
# 5. Length-beam ratio, adimensional. 
# 6. Froude number, adimensional. 

# The measured variable is the residuary resistance per unit weight of displacement: 

# 7. Residuary resistance per unit weight of displacement, adimensional. 

header = ['V_{}'.format(i) for i in range(7)]
df = pd.read_csv(folderName+fileName,sep='\s+',names=header,index_col=None)#.reset_index()

# Printing some info about the data
print("[INFO]\nNrow: {}\nNcol: {}".format(df.shape[0],df.shape[1]))
df.head(3)

[INFO]
Nrow: 308
Ncol: 7


,V_0,V_1,V_2,V_3,V_4,V_5,V_6
0,-2.3,0.568,4.78,3.99,3.17,0.125,0.11
1,-2.3,0.568,4.78,3.99,3.17,0.150,0.27
2,-2.3,0.568,4.78,3.99,3.17,0.175,0.47


In [3]:
# Setting the target column
target = df.loc[:,df.columns[-1]].values

# If necessary, dropping cols
cols = df.columns
df.drop(labels=cols[-1],axis=1,inplace=True)
df.head(3)

,V_0,V_1,V_2,V_3,V_4,V_5
0,-2.3,0.568,4.78,3.99,3.17,0.125
1,-2.3,0.568,4.78,3.99,3.17,0.150
2,-2.3,0.568,4.78,3.99,3.17,0.175


In [4]:
from sklearn.linear_model import Lasso
from sklearn import model_selection
from sklearn import metrics

In [5]:
clf = Lasso(alpha=0.3)
#             fit_intercept=True, 
#             normalize=False, 
#             precompute=False, 
#             copy_X=True, 
#             max_iter=1000, 
#             tol=0.0001, 
#             warm_start=False, 
#             positive=False, 
#             random_state=42, 
#             selection='cyclic')

## Leave-One-Out Cross Validation

In [6]:
mae_train=[]
mse_train=[]
rmse_train=[]
r2_train=[]

mae_test=[]
mse_test=[]
rmse_test=[]
mse_sum = 0
r2_test=[]
# ----------- CROSS VALIDATION ----------- #
# -------------- LeaveOneOut ----------- #
cv = model_selection.LeaveOneOut()
for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index,:], df.values[test_index,:]
    y_train, y_test = target[train_index],target[test_index]
    
    # Fitting the data into the model
    clf.fit(X_train, y_train)
    
    # FOR TRAINING
    predicted_train = clf.predict(X_train)
    mae_train.append(metrics.mean_absolute_error(y_train, predicted_train))
    mse_train.append(metrics.mean_squared_error(y_train, predicted_train))
    rmse_train.append(np.sqrt(metrics.mean_squared_error(y_train, predicted_train)))
    r2_train.append(metrics.r2_score(y_train, predicted_train))
    
    # FOR TESTING
    predicted_test = clf.predict(X_test)
    mae_test.append(metrics.mean_absolute_error(y_test, predicted_test))
    mse_test.append(metrics.mean_squared_error(y_test, predicted_test))
    mse = metrics.mean_squared_error(y_test, predicted_test)
    mse_sum = mse_sum + mse
    r2_test.append(metrics.r2_score(y_test, predicted_test))


rmse_test = np.sqrt(mse_sum/len(rmse_train))


$$MAE = \sum_{i=0}^{n} (y_i - \hat{y_i})$$

$$RMSE = \sqrt{\frac{\sum_{i=0}^{n} (y_i - \hat{y_i})^2}{n}}$$

$$RMSE = \sqrt{\frac{\sum_{i=0}^{n} (MAE_i)^2}{n}}$$

In [7]:
# Calculating the mean values for train and test scores
mae_train = np.array(mae_train).mean()
mse_train = np.array(mse_train).mean()
rmse_train = np.array(rmse_train).mean()
r2_train = np.array(r2_train).mean()

mae_test = np.array(mae_test).mean()
mse_test = np.array(mse_test).mean()
r2_test = np.array(r2_test).mean()

# Displaying the results
print('MAE Train = {}'.format(mae_train))
print('MSE Train = {}'.format(mse_train))
print('RMSE Train = {}'.format(rmse_train))
print('R2 Train = {}'.format(r2_train))
print()
print('MAE Test = {}'.format(mae_test))
print('MSE Test = {}'.format(mse_test))
print('RMSE Test = {}'.format(rmse_test))
print('R2 Test = {}'.format(r2_test))

MAE Train = 6.735170054711651
MSE Train = 87.5637012370045
RMSE Train = 9.357519663460568
R2 Train = 0.6177734062509095

MAE Test = 6.8077270525622335
MSE Test = 89.71563667769057
RMSE Test = 9.471833860329818
R2 Test = 0.0


## K-Fold Cross Validation

In [8]:
mae_train=[]
mse_train=[]
rmse_train=[]
r2_train=[]

mae_test=[]
mse_test=[]
rmse_test=[]
r2_test=[]
# ----------- CROSS VALIDATION ----------- #
# -------------- LeaveOneOut ----------- #
cv = model_selection.KFold(n_splits=5,shuffle=False)
for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index,:], df.values[test_index,:]
    y_train, y_test = target[train_index],target[test_index]
    
    # Fitting the data into the model
    clf.fit(X_train, y_train)
    
    # FOR TRAINING
    predicted_train = clf.predict(X_train)
    mae_train.append(metrics.mean_absolute_error(y_train, predicted_train))
    mse_train.append(metrics.mean_squared_error(y_train, predicted_train))
    rmse_train.append(np.sqrt(metrics.mean_squared_error(y_train, predicted_train)))
    r2_train.append(metrics.r2_score(y_train, predicted_train))
    
    # FOR TESTING
    predicted_test = clf.predict(X_test)
    mae_test.append(metrics.mean_absolute_error(y_test, predicted_test))
    mse_test.append(metrics.mean_squared_error(y_test, predicted_test))
    rmse_test.append(np.sqrt(metrics.mean_squared_error(y_test, predicted_test)))
    r2_test.append(metrics.r2_score(y_test, predicted_test))

In [9]:
# Calculating the mean values for train and test scores
mae_train = np.array(mae_train).mean()
mse_train = np.array(mse_train).mean()
rmse_train = np.array(rmse_train).mean()
r2_train = np.array(r2_train).mean()

mae_test = np.array(mae_test).mean()
mse_test = np.array(mse_test).mean()
rmse_test = np.array(rmse_test).mean()
r2_test = np.array(r2_test).mean()

# Displaying the results
print('MAE   = {}'.format(mae_train))
print('MSE Train = {}'.format(mse_train))
print('RMSE Train = {}'.format(rmse_train))
print('R2 Train = {}'.format(r2_train))
print()
print('MAE Test = {}'.format(mae_test))
print('MSE Test = {}'.format(mse_test))
print('RMSE Test = {}'.format(rmse_test))
print('R2 Test = {}'.format(r2_test))

MAE   = 6.732992216077821
MSE Train = 87.49813527588839
RMSE Train = 9.35328818551785
R2 Train = 0.6178651423359807

MAE Test = 6.7580073173701765
MSE Test = 88.42671290445486
RMSE Test = 9.370024385822669
R2 Test = 0.6143150560492299


## Hold-out Cross Validation

In [10]:
mae_train=[]
mse_train=[]
rmse_train=[]
r2_train=[]

mae_test=[]
mse_test=[]
rmse_test=[]
r2_test=[]
# ----------- CROSS VALIDATION ----------- #
# -------------- Hold-Out -------------- # When I have too many data
X_train, X_test, y_train, y_test  = model_selection.train_test_split(df.values, 
                                                                     target, 
                                                                     test_size=0.5,
                                                                     random_state=None,
                                                                     shuffle=False)
    
# Fitting the data into the model
clf.fit(X_train, y_train)

# FOR TRAINING
predicted_train = clf.predict(X_train)
mae_train.append(metrics.mean_absolute_error(y_train, predicted_train))
mse_train.append(metrics.mean_squared_error(y_train, predicted_train))
rmse_train.append(np.sqrt(metrics.mean_squared_error(y_train, predicted_train)))
r2_train.append(metrics.r2_score(y_train, predicted_train))

# FOR TESTING
predicted_test = clf.predict(X_test)
mae_test.append(metrics.mean_absolute_error(y_test, predicted_test))
mse_test.append(metrics.mean_squared_error(y_test, predicted_test))
rmse_test.append(np.sqrt(metrics.mean_squared_error(y_test, predicted_test)))
r2_test.append(metrics.r2_score(y_test, predicted_test))

In [11]:
# Calculating the mean values for train and test scores
mae_train = np.array(mae_train).mean()
mse_train = np.array(mse_train).mean()
rmse_train = np.array(rmse_train).mean()
r2_train = np.array(r2_train).mean()

mae_test = np.array(mae_test).mean()
mse_test = np.array(mse_test).mean()
rmse_test = np.array(rmse_test).mean()
r2_test = np.array(r2_test).mean()

# Displaying the results
print('MAE   = {}'.format(mae_train))
print('MSE Train = {}'.format(mse_train))
print('RMSE Train = {}'.format(rmse_train))
print('R2 Train = {}'.format(r2_train))
print()
print('MAE Test = {}'.format(mae_test))
print('MSE Test = {}'.format(mse_test))
print('RMSE Test = {}'.format(rmse_test))
print('R2 Test = {}'.format(r2_test))

MAE   = 6.512777819583015
MSE Train = 82.46400685652009
RMSE Train = 9.080969488800196
R2 Train = 0.6162623979434224

MAE Test = 6.836738858544053
MSE Test = 97.63610606004384
RMSE Test = 9.881098423760582
R2 Test = 0.598469226486182
